In [1]:
import scanpy as scp
import scipy.sparse as sprs
import pandas as pd
import warnings
import os
import numpy as np

warnings.filterwarnings("ignore")

from parameters import *

In [2]:
adataRaw = scp.read("Data/Raw.h5ad")

# Counts

In [3]:
d = pd.DataFrame(np.array(adataRaw.X.todense()))
d.to_csv(DIRECTORY+"Results/Counts_2022.csv")

# Metadata

In [4]:
metadataRaw = adataRaw.obs

In [5]:
metadata = scp.read("Analysis_Main/Results/QC.h5ad").obs
metadaCondition = []
for condition in metadata["Condition"].unique():
    metadaCondition.append(scp.read("Analysis_Main/Results/QC_"+condition+".h5ad").obs)

In [6]:
X_umap = scp.read("Analysis_Main/Results/QC.h5ad").obsm["X_umap"]
X_umap_condition = []
for condition in metadata["Condition"].unique():
    X_umap_condition.append(scp.read("Analysis_Main/Results/QC_"+condition+".h5ad").obsm["X_pca"])

In [7]:
X_pca = scp.read("Analysis_Main/Results/QC.h5ad").obsm["X_pca"]
X_pca_condition = []
for condition in metadata["Condition"].unique():
    X_pca_condition.append(scp.read("Analysis_Main/Results/QC_"+condition+".h5ad").obsm["X_pca"])

In [8]:
metadataRaw["QC_passed"] = False
metadataRaw.loc[metadata.index,"QC_passed"] = True
metadataRaw.loc[metadata.index,"leiden"] = metadata.loc[:,"leiden"]
for condition in metadaCondition:
    metadataRaw.loc[condition.index,"leiden_by_condition"] = condition.loc[:,"leiden"]
    
for i in range(2):
    metadataRaw.loc[metadata.index,"X_umap_"+str(i)] = X_umap[:,i]
for condition,umap in zip(metadaCondition,X_umap_condition):
    for i in range(2):
        metadataRaw.loc[condition.index,"X_umap_by_condition_"+str(i)] = umap[:,i]
        
for i in range(PCA):
    metadataRaw.loc[metadata.index,"X_pca_"+str(i)] = X_pca[:,i]
for condition,umap in zip(metadaCondition,X_pca_condition):
    for i in range(PCA_CONDITION):
        metadataRaw.loc[condition.index,"X_pca_by_condition_"+str(i)] = umap[:,i]

In [9]:
metadataRaw.to_csv(DIRECTORY+"Results/Cells_2022.csv")

# Metadata genes

In [10]:
genes = scp.read("Data/Raw.h5ad").var.drop("Unnamed: 0",axis=1).reset_index()

In [11]:
genes.to_csv(DIRECTORY+"Results/Genes_2022.csv")